<a href="https://colab.research.google.com/github/CalculatedContent/WeightWatcher/blob/master/examples/WW-RandMats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# WW Rand Mats

How the weightwatcher PL estimator performs on random matrices






In [1]:
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### Import WeightWatcher


In [3]:
import sys
if 'google.colab' in sys.modules:
    !pip install weightwatcher

In [4]:
import logging

import weightwatcher as ww
import torchvision.models as models

logger = logging.getLogger(ww.__name__)
logger.setLevel(logging.INFO)

ww.__version__

'0.6.5'

###  Create a Dummy Model

In [5]:
import weightwatcher as ww
import torchvision.models as model

model = models.vgg11(pretrained=True)
watcher = ww.WeightWatcher(model=model)
details = watcher.describe()

INFO:weightwatcher:

python      version 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy       version 1.19.1
tensforflow version 2.1.0
keras       version 2.2.4-tf
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': None, 'fit': 'power_law', 'sparsify': True, 'detX': True, 'mp_fit': False, 'min_evals': 0, 'max_evals': None, 'tolerance': 1e-06, 'glorot_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channe

In [6]:
details['Q']=details.N/details.M

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [7]:
details

,layer_id,name,M,N,layer_type,num_evals,rf,Q
0,2,Conv2d,3.0,64.0,LAYER_TYPE.CONV2D,27.0,9.0,21.333333
1,5,Conv2d,64.0,128.0,LAYER_TYPE.CONV2D,576.0,9.0,2.000000
2,8,Conv2d,128.0,256.0,LAYER_TYPE.CONV2D,1152.0,9.0,2.000000
3,10,Conv2d,256.0,256.0,LAYER_TYPE.CONV2D,2304.0,9.0,1.000000
4,13,Conv2d,256.0,512.0,LAYER_TYPE.CONV2D,2304.0,9.0,2.000000
5,15,Conv2d,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
6,18,Conv2d,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
7,20,Conv2d,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
8,25,Linear,4096.0,25088.0,LAYER_TYPE.DENSE,4096.0,1.0,6.125000
9,28,Linear,4096.0,4096.0,LAYER_TYPE.DENSE,4096.0,1.0,1.000000


### Q ~= 1, 2, 4, 6, 21

In [8]:
import powerlaw
Qs , alphas = [], []
for layer_id in [28, 13, 31, 25, 2]:
    Q = details[details.layer_id == layer_id].Q.to_numpy()[0]
    
    esd = watcher.get_ESD(layer=layer_id, random=True)
    fit = powerlaw.Fit(esd, xmax=np.max(esd))
    alpha = fit.alpha
    print("Q={}  alpha={:0.2}".format(Q, alpha))
    Qs.append(Q)
    alphas.append(alpha)

INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': None, 'fit': 'power_law', 'sparsify': True, 'detX': True, 'mp_fit': False, 'min_evals': 0, 'max_evals': None, 'tolerance': 1e-06, 'glorot_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': True, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': None, 'fit': 'power_law', 'sparsify': True, 'detX': True, 'mp_

Traceback (most recent call last):
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-d8f78c3fa154>", line 7, in <module>
    fit = powerlaw.Fit(esd, xmax=np.max(esd))
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/powerlaw.py", line 137, in __init__
    self.find_xmin()
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/powerlaw.py", line 236, in find_xmin
    fits = asarray(list(map(fit_function, xmins)))
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/powerlaw.py", line 233, in fit_function
    parent_Fit=self)
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/powerlaw.py", line 1113, in __init__
    Distribution.__init__(self, **kwargs)
  File "/Users/charleshmartin/anaconda3/lib/python3.7/site-packages/powerlaw.py", line 610, in __init__
    self.fit(data

TypeError: object of type 'NoneType' has no len()